<a href="https://colab.research.google.com/github/AlexxLai/ml0716/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 1s 0us/step


In [5]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [6]:
train_df = getdata("train")
test_df = getdata("test")

In [7]:
train_df

,content,sentiment
0,McConaughey in a horror/thriller? I had to see...,1
1,John Carpenter's Halloween<br /><br />Is it th...,1
2,Plot is never the strong point of a Fred Astai...,1
3,The House of the Spirits is a gripping tale of...,1
4,I loved this movie. It is a definite inspirati...,1
...,...,...
24995,"Between 1937 and 1939, Twentieth Century-Fox m...",0
24996,"Some might scoff, but there is actually a real...",0
24997,If you look at Corey Large's information here ...,0
24998,<br /><br />Worst. Movie. Ever.<br /><br />Wha...,0


In [11]:
# Tokenizer: 轉換成數字
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [ ]:
# 0不要用掉,0是用來padding
tok.word_index

In [12]:
# sequences: 數字序列
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,8,3,186,705,10,66,5,64,11.0,10.0,13.0,767.0,7.0,7.0,1.0,111.0,6.0,576.0,8.0,2753.0,2.0,9.0,32.0,894.0,1276.0,2.0,651.0,220.0,4.0,286.0,167.0,140.0,3.0,52.0,1158.0,10.0,188.0,132.0,73.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,304,2186,7,7,6,9,1,830,186.0,19.0,4.0,29.0,55.0,54.0,276.0,21.0,5.0,313.0,18.0,5.0,69.0,9.0,6.0,2.0,207.0,77.0,27.0,1.0,19.0,6.0,2095.0,1257.0,2.0,77.0,207.0,1067.0,3.0,52.0,315.0,270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,111,6,112,1,562,210,4,3,1794.0,2.0,17.0,18.0,790.0,1.0,878.0,6.0,1495.0,1794.0,2.0,128.0,213.0,122.0,29.0,205.0,33.0,294.0,2.0,1588.0,622.0,962.0,2.0,1618.0,1051.0,51.0,1.0,51.0,33.0,906.0,171.0,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,310,4,1,6,3,783,4,220.0,1220.0,295.0,2410.0,2.0,1160.0,1245.0,1718.0,488.0,2.0,718.0,664.0,5.0,110.0,16.0,29.0,91.0,1795.0,2.0,833.0,11.0,8.0,58.0,647.0,6.0,28.0,4.0,1.0,115.0,105.0,4.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10,444,11,17,9,6,3,17,9.0,22.0,16.0,11.0,17.0,6.0,287.0,1.0,2353.0,39.0,287.0,2640.0,9.0,141.0,27.0,8.0,341.0,115.0,17.0,10.0,25.0,107.0,8.0,3.0,193.0,55.0,9.0,77.0,94.0,22.0,1165.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,197,2,1114,1675,90,3,4,440,105.0,187.0,947.0,1.0,127.0,4.0,1.0,198.0,9.0,13.0,574.0,12.0,1.0,1178.0,66.0,1.0,35.0,5.0,1125.0,11.0,12.0,3.0,245.0,120.0,45.0,2116.0,91.0,2471.0,2.0,1.0,8.0,2908.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,46,235,18,47,6,162,3,144,495.0,16.0,228.0,569.0,75.0,105.0,11.0,43.0,20.0,29.0,7.0,7.0,3.0,759.0,4.0,182.0,81.0,369.0,16.0,2.0,827.0,1953.0,137.0,5.0,19.0,538.0,2.0,127.0,53.0,617.0,80.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,44,22,165,30,1616,130,20,895,681.0,222.0,3.0,17.0,443.0,8.0,362.0,14.0,4.0,8.0,60.0,237.0,393.0,3.0,106.0,769.0,7.0,7.0,83.0,4.0,29.0,86.0,124.0,138.0,3.0,2135.0,17.0,123.0,3.0,752.0,2.0,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,7,7,246,17,123,7,7,48,13.0,1.0,1286.0,4.0,1421.0,11.0,1031.0,1209.0,36.0,1583.0,9.0,80.0,3.0,1550.0,784.0,15.0,1.0,1322.0,812.0,1143.0,12.0,518.0,10.0,479.0,1.0,164.0,112.0,293.0,1.0,201.0,348.0,105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen = 512)
x_test_pad = pad_sequences(x_test_seq, maxlen = 512)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,227,59,22,137,5,205,3,352,2,48,6,205,550,143,128,533,7,7,2,1339,10,383,690,155,2,11,6,28,4,145,99,12,1011,2,3,330,39,57,837,825
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,9,53,10,1461,22,64,11,19,30,219,277,8,126,110,14,9,6,3,8,19,228,2,6,206,3,822,44,21,1,830,92,28,4,1,830,186,105,4,29,55
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,7,7,9,59,27,3,5,866,140,88,4,1,411,4,790,1,171,2,8,189,42,21,1664,85,1,111,1706,1,618,1391,18,10,97,103,1,687,117,2,117
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,220,8,3,2754,1177,488,14,796,405,3,52,724,236,1,4,6,3,15,3,696,295,701,2767,4,341,2,23,1127,2,1,807,175,1876,758,1777,5,1,1203,322,838
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,193,17,18,172,55,10,103,9,10,112,1492,12,9,6,14,193,14,9,6,10,76,35,8,9,12,9,268,35,1600,10,116,11,17,10,103,9,1939,10,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,441,1,19,6,1748,18,6,79,2314,12,1,198,141,25,1051,130,16,1,105,423,25,73,1204,550,14,257,3,2259,857,970,106,328,584,25,74,8,1,175,39,8
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,53,617,80,3,2018,7,7,21,61,13,1,537,1212,18,46,4,1,1492,1,2073,1,8,840,165,37,2033,213,43,4,7,7,54,49,36,377,5,1360,89,64,9
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,18,441,9,465,63,813,236,13,15,69,28,4,1,528,4,1,19,143,79,3,224,767,30,29,1,81,20,1,305,536,234,10,194,1,522,393,13,181,18,21
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1143,12,518,10,479,1,164,112,293,1,201,348,105,60,120,36,2367,1841,65,578,7,7,2,86,615,119,1,121,60,2073,8,1,5,1,1029,1227,7,7,297,155


In [31]:
import numpy as np
y_train = np.array(train_df["sentiment"])
y_test = np.array(test_df["sentiment"])

In [32]:
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
INPUT_DIM = 3001
OUTPUT_DIM = 128
INPUT_LENGTH = 512
layers = [
    Embedding(INPUT_DIM, OUTPUT_DIM, mask_zero=True, input_length=INPUT_LENGTH),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 512, 128)          384128    
_________________________________________________________________
flatten_2 (Flatten)          (None, 65536)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 256)               16777472  
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 514       
Total params: 17,162,114
Trainable params: 17,162,114
Non-trainable params: 0
_________________________________________________________________


In [33]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [34]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("embedding.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)       
]
model.fit(x_train_pad, 
     y_train, 
     batch_size=200,
     validation_split=0.1,
     epochs=50,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 4s - loss: 0.7014 - accuracy: 0.5911 - val_loss: 0.6433 - val_accuracy: 0.6232
Epoch 2/50
113/113 - 4s - loss: 0.3238 - accuracy: 0.8637 - val_loss: 0.3630 - val_accuracy: 0.8388
Epoch 3/50
113/113 - 4s - loss: 0.1731 - accuracy: 0.9387 - val_loss: 0.3470 - val_accuracy: 0.8648
Epoch 4/50
113/113 - 4s - loss: 0.0693 - accuracy: 0.9810 - val_loss: 0.4698 - val_accuracy: 0.8416
Epoch 5/50
113/113 - 4s - loss: 0.0167 - accuracy: 0.9975 - val_loss: 0.4504 - val_accuracy: 0.8736
Epoch 6/50
113/113 - 4s - loss: 0.0045 - accuracy: 0.9997 - val_loss: 0.7590 - val_accuracy: 0.8288
Epoch 7/50
113/113 - 4s - loss: 0.0015 - accuracy: 0.9999 - val_loss: 0.6945 - val_accuracy: 0.8520
Epoch 8/50
113/113 - 4s - loss: 6.7201e-04 - accuracy: 1.0000 - val_loss: 0.9879 - val_accuracy: 0.8108


In [35]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.3111 - accuracy: 0.8745


[0.31105536222457886, 0.8744800090789795]